In [ ]:
# ===== Start Screen ======

# 背景音樂載入並播放（無限循環）
pygame.mixer.music.load("sounds/backgroundmusic.wav")
pygame.mixer.music.set_volume(0.5)   # 音量（0.0 ~ 1.0）
pygame.mixer.music.play(-1)          # -1 表示無限循環播放

# 顏色定義
WHITE = (255, 255, 255)
BLUE = (0, 120, 215)
LIGHT_BLUE = (0, 150, 255)
DARK_BLUE = (86,105,179)
NIGHT_BUTTON_COLOR = (200, 200, 200)
NIGHT_BUTTON_HOVER = (100, 100, 100)

# 字型
font = pygame.font.SysFont(None, 48)

# 螢幕設定
info = pygame.display.Info()
WIDTH, HEIGHT = info.current_w, info.current_h
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Cashflow Master")

# 圖片載入
background_day = pygame.transform.scale(pygame.image.load("images/background/background_daylight.png"), (WIDTH, HEIGHT))
background_night = pygame.transform.scale(pygame.image.load("images/background/background_night.png"), (WIDTH, HEIGHT))
road_day = pygame.image.load("images/road/road_daylight.png")
road_night = pygame.image.load("images/road/road_night.png")

walk1 = pygame.image.load("images/walk/walk1.png")
walk2 = pygame.image.load("images/walk/walk2.png")
walk3 = pygame.image.load("images/walk/walk3.png")
walk4 = pygame.image.load("images/walk/walk4.png")
walk1 = pygame.transform.scale(walk1, (walk1.get_width() // 2, walk1.get_height() // 2))
walk2 = pygame.transform.scale(walk2, (walk2.get_width() // 2, walk2.get_height() // 2))
walk3 = pygame.transform.scale(walk3, (walk3.get_width() // 2, walk3.get_height() // 2))
walk4 = pygame.transform.scale(walk4, (walk4.get_width() // 2, walk4.get_height() // 2))

logo_img = pygame.image.load("images/logo.png")
nightmode_icon = pygame.image.load("images/nightmode.png")
nightmode_icon = pygame.transform.scale(nightmode_icon, (60, 60))
nightmode_rect = nightmode_icon.get_rect(topleft=(1500, 900))

class Character:
    def __init__(self, x, y):
        self.images = [walk1, walk2, walk3, walk4]
        self.index = 0
        self.timer = 0
        self.fps = 60
        self.frame_rate = 8
        self.delay = self.fps // self.frame_rate
        self.x = -200
        self.y = 300

    def update(self):
        self.timer += 1
        if self.timer >= self.delay:
            self.index = (self.index + 1) % len(self.images)
            self.timer = 0

    def draw(self, surface):
        surface.blit(self.images[self.index], (self.x, self.y))

class Road:
    def __init__(self, img, y):
        self.img = img
        self.y = y
        self.x1 = 0
        self.x2 = img.get_width()
        self.speed = 2

    def set_image(self, img):
        self.img = img
        self.x1 = 0
        self.x2 = img.get_width()

    def update(self):
        self.x1 -= self.speed
        self.x2 -= self.speed
        if self.x1 <= -self.img.get_width():
            self.x1 = self.x2 + self.img.get_width()
        if self.x2 <= -self.img.get_width():
            self.x2 = self.x1 + self.img.get_width()

    def draw(self, surface):
        surface.blit(self.img, (self.x1, self.y))
        surface.blit(self.img, (self.x2, self.y))

class Button:
    def __init__(self, rect, text):
        self.rect = rect
        self.text = text

    def draw(self, surface, is_night):
        mouse_pos = pygame.mouse.get_pos()
        if is_night:
            color = NIGHT_BUTTON_HOVER if self.rect.collidepoint(mouse_pos) else NIGHT_BUTTON_COLOR
        else:
            color = LIGHT_BLUE if self.rect.collidepoint(mouse_pos) else BLUE

        pygame.draw.rect(surface, color, self.rect, border_radius=12)
        text_surf = font.render(self.text, True, WHITE)
        text_rect = text_surf.get_rect(center=self.rect.center)
        surface.blit(text_surf, text_rect)

    def is_clicked(self, event):
        return event.type == pygame.MOUSEBUTTONDOWN and self.rect.collidepoint(event.pos)

# 初始化元素
is_night = False
road_y = HEIGHT - road_day.get_height() + 200
character = Character(300, road_y - walk1.get_height() + 10)
road = Road(road_day, road_y)
logo_rect = logo_img.get_rect(midtop=(WIDTH // 2, -150))
button = Button(pygame.Rect(WIDTH // 2 - 100, HEIGHT // 2, 200, 60), "Start Game")

clock = pygame.time.Clock()
running = True
game_started = False

while running:
    dt = clock.tick(60)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if not game_started and button.is_clicked(event):
            click_sound.play()
            pygame.mixer.music.stop()  # 停止背景音樂
            pygame.quit()              # 關閉 Pygame
            # 執行 main_game.py，假設 python 執行環境是 python3
            subprocess.run(["python3", "main_game.py"])  
            sys.exit()                 # 退出此程式

        if event.type == pygame.MOUSEBUTTONDOWN and nightmode_rect.collidepoint(event.pos):
            click_sound.play()
            is_night = not is_night
            road.set_image(road_night if is_night else road_day)
            
    current_background = background_night if is_night else background_day

    if not game_started:
        road.update()
        character.update()
        screen.blit(current_background, (0, 0))
        road.draw(screen)
        character.draw(screen)
        screen.blit(logo_img, logo_rect)
        button.draw(screen, is_night)
        screen.blit(nightmode_icon, nightmode_rect)

        mouse_pos = pygame.mouse.get_pos()
        if nightmode_rect.collidepoint(mouse_pos):
            tooltip_visible = True
            tooltip_alpha = min(255, tooltip_alpha + 15)
        else:
            tooltip_alpha = max(0, tooltip_alpha - 15)
            if tooltip_alpha == 0:
                tooltip_visible = False

        if tooltip_alpha > 0:
            tooltip_text = font.render("Night Mode", True, (255, 255, 255))
            text_rect = tooltip_text.get_rect(center=(nightmode_rect.centerx, nightmode_rect.top - 40))

            bubble_padding = 10
            bubble_rect = pygame.Rect(
                text_rect.left - bubble_padding,
                text_rect.top - bubble_padding,
                text_rect.width + 2 * bubble_padding,
                text_rect.height + 2 * bubble_padding
            )

            tooltip_surf = pygame.Surface((bubble_rect.width, bubble_rect.height), pygame.SRCALPHA)
            pygame.draw.rect(
                tooltip_surf,
                (0, 0, 0, tooltip_alpha),
                tooltip_surf.get_rect(),
                border_radius=10
            )

            screen.blit(tooltip_surf, (bubble_rect.x, bubble_rect.y))
            text_surf_alpha = tooltip_text.copy()
            text_surf_alpha.set_alpha(tooltip_alpha)
            screen.blit(text_surf_alpha, text_rect)

    else:
        # 不需要用這段，因為按 Start Game 就會執行 main_game.py 並退出此程式
        pass

    pygame.display.flip()

pygame.quit()
sys.exit()
